<a href="https://colab.research.google.com/github/yasabuu/Anime-Forcasting/blob/main/Testing_Fuzzy_Matching_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process

In [ ]:
# data to dataframe
# netflix data

#anilist data

In [ ]:
#first we import libraries to use
import pandas as pd
from google.colab import auth
import pandas_gbq

# Will collect credentials
auth.authenticate_user()

In [ ]:
#Query Bigquery
query = "SELECT * FROM `anime-forecasting.anime_datasets.cleaned-netflix-anime-only-2023`"
netflix_df_23 = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


In [ ]:
netflix_df_23.isnull().sum()

,0
Title,0
title_split,0
title_split_characters,92
title_split_characters_extra,2098
Available_Globally_,0
Release_Date,2088
Hours_Viewed,0
Runtime,2239
Views,2239
netflix_year,0


In [ ]:
#Query Bigquery
query = "SELECT * FROM `anime-forecasting.anime_datasets.cleaned-netflix-anime-only-2024-3`"
netflix_df_24 = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


In [ ]:
netflix_df_24.rename(columns={'split': 'title_split'}, inplace=True)
netflix_df_24.rename(columns={'split_characters': 'title_split_characters'}, inplace=True)

In [ ]:
netflix_df_24.shape

(24759, 7)

In [ ]:
netflix_df_24.isnull().sum()

,0
Title,0
title_split,2405
title_split_characters,17271
Available_Globally_,0
Release_Date,16605
Hours_Viewed,0
netflix_year,0


In [ ]:
netflix_df_24 = netflix_df_24.dropna(subset=['title_split'])

In [ ]:
netflix_df_23['netflix_year'] = 2023

# Step 2: Add a new column `netflix_year` and set to 2024 for netflix_df_24
netflix_df_24['netflix_year'] = 2024

netflix_df = pd.concat([netflix_df_24, netflix_df_23])

In [ ]:
netflix_df.isnull().sum()

,0
Title,0
title_split,2405
title_split_characters,17363
Available_Globally_,0
Release_Date,18693
Hours_Viewed,0
netflix_year,0
title_split_characters_extra,26857
Runtime,26998
Views,26998


In [ ]:
#Query Bigquery
query = "SELECT * FROM `anime-forecasting.Anilist_dataset.anilist_main_table`"
anilist_df = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


In [ ]:
anilist_df.isnull().sum()

,0
id,0
idMal,0
title,0
format,0
episodes,0
duration,0
status,0
startDay,0
startMonth,0
startYear,0


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

# Function to find the closest match from anilist_titles using RapidFuzz
def match_title(row, anilist_titles, anilist_ids):
    title_to_match = row['title_split']
    match, score, index = process.extractOne(title_to_match, anilist_titles, scorer=fuzz.ratio)

    if score > 90:  # Adjust the threshold as needed
          return anilist_ids[index]  # Return the corresponding id from anilist_ids
    else:
          return None
# Create a list of titles and ids from anilist_df for matching
anilist_titles = anilist_df['title'].tolist()  # Titles from anilist_df
anilist_ids = anilist_df['id'].tolist()  # Corresponding ids from anilist_df

# Apply the match_title function to each row in netflix_df to find and assign the id
netflix_df['id'] = netflix_df.apply(match_title, anilist_titles=anilist_titles, anilist_ids=anilist_ids, axis=1)


In [ ]:
netflix_df.shape

In [ ]:
#Query Bigquery
query = "SELECT * FROM `anime-forecasting.Kaggle_dataset.Kaggle_data`"
kaggle_df = pandas_gbq.read_gbq(query, project_id="anime-forecasting")

Downloading: 100%|██████████|


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

# Create a function to find the closest match using rapidfuzz
def match_title(row, kaggle_titles, kaggle_ids):
    title_to_match = row['title_split']
    match, score, index = process.extractOne(title_to_match, kaggle_titles, scorer=fuzz.ratio)

    # Return the corresponding anime_id if match is good enough (e.g. score > 80)
    if score > 90:  # Adjust the threshold as needed
        return kaggle_ids[index]
    else:
        return None

# Create a list of titles and anime_ids from kaggle_df for faster processing
kaggle_titles = kaggle_df['Name'].tolist()
kaggle_ids = kaggle_df['anime_id'].tolist()

# Apply the matching function to netflix_df
netflix_df['anime_id'] = netflix_df.apply(match_title, kaggle_titles=kaggle_titles, kaggle_ids=kaggle_ids, axis=1)

# Now, netflix_df will have a new column 'anime_id' with the matched anime_id or None if no match was found.

In [ ]:
netflix_df

,Title,title_split,title_split_characters,Available_Globally_,Release_Date,Hours_Viewed,netflix_year,title_split_characters_extra,Runtime,Views,anime_id,id
0,The Night Agent: Season 1,The Night Agent: Season 1,None,True,2023-03-23,812100000,2024,NaN,NaN,<NA>,NaN,NaN
1,Ginny & Georgia: Season 2,Ginny & Georgia: Season 2,None,True,2023-01-05,665100000,2024,NaN,NaN,<NA>,NaN,NaN
2,The Glory: Season 1 // 더 글로리: 시즌 1,The Glory: Season 1,더 글로리: 시즌 1,True,2022-12-30,622800000,2024,NaN,NaN,<NA>,NaN,NaN
3,Wednesday: Season 1,Wednesday: Season 1,None,True,2022-11-23,507700000,2024,NaN,NaN,<NA>,NaN,NaN
4,Queen Charlotte: A Bridgerton Story,Queen Charlotte: A Bridgerton Story,None,True,2023-05-04,503000000,2024,NaN,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2424,Eternal Yesterday: Season 1 // 永遠の昨日: シーズン1,Eternal Yesterday: Season 1,永遠の昨日: シーズン1,False,NaT,200000,2023,None,3:13:00,100000,NaN,NaN
2425,Hi! Come in: Season 3 // 嗨! 營業中: 第 3 季,Hi! Come in: Season 3,嗨! 營業中: 第 3 季,False,NaT,200000,2023,None,3:10:00,100000,NaN,NaN
2426,There Is No Other: Season 1 // 夢中さ、きみに。: シーズン1,There Is No Other: Season 1,夢中さ、きみに。: シーズン1,False,NaT,200000,2023,None,2:00:00,100000,NaN,NaN
2427,Detective Conan - Police school edition Wild P...,Detective Conan - Police school edition Wild P...,名探偵コナン 警察学校編: シーズン1,False,NaT,100000,2023,None,1:38:00,100000,NaN,NaN


In [ ]:
netflix_df = netflix_df.drop(['title_split_characters', 'title_split_characters_extra'], axis=1)

In [ ]:
no_null_anilist_netflix_df = netflix_df.dropna(subset=['anime_id'])

In [ ]:
no_null_kaggle_netflix_df = netflix_df.dropna(subset=['id'])

In [ ]:
no_nulls_netflix_df = netflix_df.dropna(subset=['anime_id'])
no_null_ids_netflix_df = no_nulls_netflix_df.dropna(subset=['id'])

In [ ]:
no_null_ids_netflix_df.isnull().sum()

,0
Title,0
title_split,0
Available_Globally_,0
Release_Date,99
Hours_Viewed,0
netflix_year,0
Runtime,102
Views,102
anime_id,0
id,0


In [ ]:
#merged_df = pd.merge(filtered_df, kaggle_df, on='anime_id', how='inner')

In [ ]:
#selected_columns_df = merged_df[['title_split', 'anime_id', 'Name']]

In [ ]:
#selected_columns_df.to_csv("checking_rapid_fuzz.csv", index=False)

In [ ]:
#selected_columns_df.shape

In [ ]:
#netflix_df['anime_id'].notnull().sum()

In [ ]:
#selected_columns_df.to_csv("checking_rapid_fuzz.csv", index=False)

In [ ]:
#netflix_df.to_csv("netflix_with_anime_id_kaggle.csv", index=False)

In [ ]:
no_null_anilist_netflix_df.to_csv("no_null_anilist_netflix_df.csv", index=False)

In [ ]:
no_null_kaggle_netflix_df.to_csv("no_null_kaggle_netflix_df.csv", index=False)

In [ ]:
no_null_ids_netflix_df.to_csv("no_null_ids_netflix_df.csv", index=False)